In [1]:
from classifier import train_classifier
from features import extract_features, flatten_features
from utils.dataset import read_dataset
from utils.preprocessing import preprocess_dataframe, extract_labels, decipher_labels, oversample_minority_classes
from utils.scoring import print_cv_score, evaluate_submission
from utils.splits import generate_hold_out_split

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [2]:
raw_data = read_dataset('data')
data = preprocess_dataframe(raw_data, 'raw_data')
labels = extract_labels(data)

In [3]:
features = extract_features(data, raw_data)

In [4]:
flattened_features = flatten_features(features)

In [5]:
training_data, testing_data, unused_data = generate_hold_out_split(raw_data)
training_features, testing_features = flattened_features.iloc[training_data.index], flattened_features.iloc[testing_data.index]
training_labels, testing_labels = labels.iloc[training_data.index], labels.iloc[testing_data.index]

In [6]:
oversampled_training_features, oversampled_training_labels = oversample_minority_classes(training_features, training_labels)

Oversampling agree group...
Oversampling disagree group...
Oversampling discuss group...


In [7]:
classifier = train_classifier(oversampled_training_features, oversampled_training_labels)

In [8]:
print_cv_score(classifier, training_features, training_labels, cv=5)

Cross validation scores: [ 0.83378747  0.8773842   0.84412033  0.84854015  0.86488971], AVG - 0.853744368432165


In [9]:
predictions = testing_data.copy()
predictions['Stance'] = decipher_labels(classifier.predict(testing_features), index=testing_features.index)
evaluate_submission(testing_data, predictions)

CONFUSION MATRIX:
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    146    |     6     |    178    |    26     |
-------------------------------------------------------------
| disagree  |    45     |     1     |    42     |     3     |
-------------------------------------------------------------
|  discuss  |    188    |     0     |    717    |    104    |
-------------------------------------------------------------
| unrelated |     6     |     0     |    32     |   4222    |
-------------------------------------------------------------
ACCURACY: 0.890

MAX  - the best possible score (100% accuracy)
NULL - score as if all predicted stances were unrelated
TEST - score based on the provided predictions

||    MAX    ||    NULL   ||    TEST   ||
||  2521.0   ||  1065.0   ||  2034.25  ||

80.6921856406188% score achieved
